In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
import csv
from random import randrange
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numpy import float32
import warnings
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')

In [ ]:
def multiply(a,b):
    sum_ab=0.0
    for i in range(len(a)):
        temp=a[i]*b[i]
        sum_ab+=temp
    return sum_ab

In [ ]:
def cal_pearson(x,y):
    n=len(x)
    sum_x=sum(x)
    sum_y=sum(y)
    sum_xy=multiply(x,y)
    sum_x2 = sum([pow(i,2) for i in x])
    sum_y2 = sum([pow(j,2) for j in y])
    molecular=sum_xy-(float(sum_x)*float(sum_y)/n)
    denominator=sqrt((sum_x2-float(sum_x**2)/n)*(sum_y2-float(sum_y**2)/n))
    return molecular/denominator

In [ ]:
def feature_ranking(pearson):
    idx = np.argsort(pearson)  
    return idx[::-1]

In [ ]:
acc_list =list()

def main():
    expression=np.loadtxt('初筛后的基因shu.csv',delimiter=',')
    X=np.array(expression[:,:-1],dtype=float32)
    y=np.array(expression[:,-1], dtype=int)
    n_samples, n_features = X.shape    # number of samples and number of features
    kf =KFold(n_splits=10)
    
    feature_number_list =list(range(5,21,1))+list(range(30,101,10))
    
    for feature_number in feature_number_list:
        clf = RandomForestClassifier(n_estimators=200,max_features='sqrt', oob_score=True,random_state=120) 
        #clf = xgb.XGBClassifier(objective= 'multi:softmax',num_class=6,nthread=4,seed=10,eval_metric='merror')
        #clf = LogisticRegression(multi_class="multinomial",solver="newton-cg")   
        correct = 0
        for train, test in kf.split(X):
            pearson =list()
            for i in range(n_features):
                pearson.append(cal_pearson(X[train,i],y[train]))
            idx = feature_ranking(pearson)  
            selected_features = X[:, idx[0:feature_number]] 
            clf.fit(selected_features[train], y[train])  # train a classification model with the selected features on the training dataset
            y_predict = clf.predict(selected_features[test])
            acc = accuracy_score(y[test], y_predict)
            correct = correct + acc
        acc_list.append(float(correct)/10)
        print('Accuracy:{:6f}'.format(float(correct)/10))
    return acc_list

main()